In [11]:
%cd /content/
%load_ext autoreload
%autoreload 2
%matplotlib inline
from exp.nb_06 import *

/content


In [12]:
def get_data():
    path=Path('/content/mnist.pkl.gz')
    with gzip.open(path) as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
    return map(tensor, (x_train, y_train, x_valid, y_valid))

In [13]:
x_train, y_train, x_valid, y_valid = get_data()

In [14]:
x_train.shape, y_train.shape, x_valid.shape, y_valid.shape

(torch.Size([50000, 784]),
 torch.Size([50000]),
 torch.Size([10000, 784]),
 torch.Size([10000]))

In [ ]:
import inspect
print(inspect.getsource(view_tfm))
print(inspect.getsource(DataBunch))
print(inspect.getsource(get_dls))

def view_tfm(*size):
    def _inner(x): return x.view(*((-1,)+size))
    return _inner

class DataBunch():
    def __init__(self, train_dl, valid_dl, c=None):
        self.train_dl,self.valid_dl,self.c = train_dl,valid_dl,c

    @property
    def train_ds(self): return self.train_dl.dataset

    @property
    def valid_ds(self): return self.valid_dl.dataset

def get_dls(train_ds, valid_ds, bs, **kwargs):
    return (DataLoader(train_ds, batch_size=bs, shuffle=True, **kwargs),
            DataLoader(valid_ds, batch_size=bs*2, **kwargs))



In [15]:
train_ds, valid_ds = Dataset(x_train, y_train), Dataset(x_valid, y_valid)
bs, nh = 512, 50
c = y_train.max()+1
loss_func = F.cross_entropy
data = DataBunch(*get_dls(train_ds, valid_ds, bs), c)

In [32]:
mnist_view = view_tfm(1, 28, 28)

In [40]:
# make view_tfm2 using functools.partial
from functools import partial
def view_tfm2(size, x): return x.view(*((-1,) + size))
partial(view_tfm2, (1, 28, 28)) # this has same functionality with `view_tfm(1, 28, 28)`

torch.Size([1, 1, 28, 28])

In [41]:
cbfs = [Recorder,
        partial(AvgStatsCallback, accuracy),
        partial(BatchTransformXCallback, mnist_view)]

nfs = [8, 16, 32, 64, 64]

learn, run = get_learn_run(nfs, data, 0.4, conv_layer, cbs=cbfs)

In [ ]:
run.fit(5, learn)

train: [1.518158125, tensor(0.5078)]
valid: [0.385849609375, tensor(0.8831)]
train: [0.25443435546875, tensor(0.9206)]
valid: [0.1566339599609375, tensor(0.9541)]
train: [0.138890654296875, tensor(0.9570)]
valid: [0.12315595703125, tensor(0.9618)]
train: [0.103854560546875, tensor(0.9673)]
valid: [0.0992807861328125, tensor(0.9702)]
train: [0.08531361328125, tensor(0.9734)]
valid: [0.0877613037109375, tensor(0.9732)]


In [ ]:
print(inspect.getsource(conv_layer))
print(inspect.getsource(get_cnn_model))
print(inspect.getsource(get_cnn_layers))
print(inspect.getsource(get_learn_run))

def conv_layer(ni, nf, ks=3, stride=2, **kwargs):
    return nn.Sequential(
        nn.Conv2d(ni, nf, ks, padding=ks//2, stride=stride), GeneralRelu(**kwargs))

def get_cnn_model(data, nfs, layer, **kwargs):
    return nn.Sequential(*get_cnn_layers(data, nfs, layer, **kwargs))

def get_cnn_layers(data, nfs, layer, **kwargs):
    nfs = [1] + nfs
    return [layer(nfs[i], nfs[i+1], 5 if i==0 else 3, **kwargs)
            for i in range(len(nfs)-1)] + [
        nn.AdaptiveAvgPool2d(1), Lambda(flatten), nn.Linear(nfs[-1], data.c)]

def get_learn_run(nfs, data, lr, layer, cbs=None, opt_func=None, uniform=False, **kwargs):
    model = get_cnn_model(data, nfs, layer, **kwargs)
    init_cnn(model, uniform=uniform)
    return get_runner(model, data, lr=lr, cbs=cbs, opt_func=opt_func)



In [ ]:
print(inspect.getsource(flatten))

def flatten(x):      return x.view(x.shape[0], -1)



In [ ]:
learn.model

Sequential(
  (0): Sequential(
    (0): Conv2d(1, 8, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
    (1): GeneralRelu()
  )
  (1): Sequential(
    (0): Conv2d(8, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): GeneralRelu()
  )
  (2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): GeneralRelu()
  )
  (3): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): GeneralRelu()
  )
  (4): Sequential(
    (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): GeneralRelu()
  )
  (5): AdaptiveAvgPool2d(output_size=1)
  (6): Lambda()
  (7): Linear(in_features=64, out_features=10, bias=True)
)

---

- A1

In [ ]:
class Lambda(nn.Module):
    def __init__(self, func):
        self.func=func
    def forward(self, x):
        return self.func(x)

- A2

In [ ]:
def mnist_resize(x): return x.view(-1, 1, 28, 28)

def flatten(x): return x.view(x.shape[0], -1)
